# Avnatis Mapping Instructions

The following describes the process for mapping the current Avantis Classes to the new Classification system. 

## SQL connection
The first step connects to the Avantis SQL server to get the list of entities

In [11]:
import pyodbc
import pandas as pd
import os
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set_theme()

connect = 'DSN=Avantis6-P;UID='+ os.environ['Avantis_User'] + ';PWD=' + os.environ['Avantis_Pass']
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connect})

engine = create_engine(connection_url)

SQL1 = """SELECT Distinct MAINTENT.id as [Entity_number],
[MAINTENT].[aenm] as [Description],
MAINTENT2.id as [Parent],
MAINTENT2.aenm as [Parent_Description],
[contname] as [Category],
[entclsid] as [Class],
SUSPEND.suspoi as [Suspended]

FROM  [AvantisP].[mc].[MAINTENT]
	  Left Join [AvantisP].[mc].[ENTCLASS] on MAINTENT.entclsref_oi = ENTCLASS.entcloi
	  Left Join [AvantisP].[mc].CATVAL ON MAINTENT.cat1_oi = CATVAL.cvoi
	  Left Join MC.SUSPEND ON MAINTENT.susp_oi = SUSPEND.suspoi
	  Left Join MC.MELINK ON MAINTENT.mtnoi = MELINK.mtnchild_oi
	  Left Join MC.MAINTENT MAINTENT2 ON MELINK.mtnparn_oi = MAINTENT2.mtnoi
WHERE
    (SUSPEND.audt_updted_dttm <= '1900-01-01 23:59:59' OR SUSPEND.audt_updted_dttm IS NULL)
"""


df = pd.read_sql(SQL1,engine)
df = df[df['Entity_number'].values != None]
df

,Entity_number,Description,Parent,Parent_Description,Category,Class,Suspended
0,\tFCL_ELS_CBL_001L,"Electrical Power Line,4.16KV,From BUS-00B1-A t...",FCL_ELS_4.16KV_LINES,"Electrical Power Line,4.16KV",Air Handling Unit,Electrical Power Line,None
1,\tFCL_ELS_CBL_002D,"Electrical Power Line,4.16KV,From BUS-00B2-A t...",FCL_ELS_4.16KV_LINES,"Electrical Power Line,4.16KV",Air Handling Unit,Electrical Power Line,None
2,\tFCL_ELS_CBL_002F,"Electrical Power Line,4.16KV,From BUS-00B2 to ...",FCL_ELS_4.16KV_LINES,"Electrical Power Line,4.16KV",Air Handling Unit,Electrical Power Line,None
3,\tFCL_ELS_CBL_002H,"Electrical Power Line,4.16KV,From BUS-00B1 to ...",FCL_ELS_4.16KV_LINES,"Electrical Power Line,4.16KV",Air Handling Unit,Electrical Power Line,None
4,\tFCL_ELS_CBL_002L,"Electrical Power Line,4.16KV,From BUS-00B2-A t...",FCL_ELS_4.16KV_LINES,"Electrical Power Line,4.16KV",Air Handling Unit,Electrical Power Line,None
...,...,...,...,...,...,...,...
126013,YTF-VSTORE,Computer Storage Location,YTF EUS,Production & End User Support,None,None,None
126014,YX2411A,"Pump, Unwatering, Primary Tanks 1-2",TAB-PRM-P-SUMP,"P Bldg & Old PS Buildings, Primary Treatment S...","Pump,Centrifugal,single-Stage",Pump,None
126015,ZB20-CA-31,"Damper,Combustion Air, Steam Boiler #20",TAB-ACC-BO-0500,"Boiler #20, Volcano",Damper/Louver,Damper/Louver,None
126016,ZXDATA-PILOT1,"DO NOT USE - City of Toronto Pump, RW Pump 18",None,None,"Pump,Centrifugal",Pump,None


Here is were we capture the 13040 tag section if it is present in the entity number

In [12]:
import re

df_13040 = pd.read_excel('13040 Codes.xlsx', sheet_name='Append1')
dict_13040 = dict(zip(df_13040['CODE'], df_13040['DESCRIPTION']))

def extract_code(entity):
    # Use regex to match the third group between dashes
    match = re.match(r'^[^-]+-[^-]+-([^-]+)-[^-]+$', str(entity))
    return match.group(1) if match else None

def code_in_dict(code):
    return code if code in dict_13040 else None

df['Extracted_Code'] = df['Entity_number'].apply(extract_code)
df['Matched_Code'] = df['Extracted_Code'].apply(code_in_dict)
df

,Entity_number,Description,Parent,Parent_Description,Category,Class,Suspended,Extracted_Code,Matched_Code
0,\tFCL_ELS_CBL_001L,"Electrical Power Line,4.16KV,From BUS-00B1-A t...",FCL_ELS_4.16KV_LINES,"Electrical Power Line,4.16KV",Air Handling Unit,Electrical Power Line,None,None,None
1,\tFCL_ELS_CBL_002D,"Electrical Power Line,4.16KV,From BUS-00B2-A t...",FCL_ELS_4.16KV_LINES,"Electrical Power Line,4.16KV",Air Handling Unit,Electrical Power Line,None,None,None
2,\tFCL_ELS_CBL_002F,"Electrical Power Line,4.16KV,From BUS-00B2 to ...",FCL_ELS_4.16KV_LINES,"Electrical Power Line,4.16KV",Air Handling Unit,Electrical Power Line,None,None,None
3,\tFCL_ELS_CBL_002H,"Electrical Power Line,4.16KV,From BUS-00B1 to ...",FCL_ELS_4.16KV_LINES,"Electrical Power Line,4.16KV",Air Handling Unit,Electrical Power Line,None,None,None
4,\tFCL_ELS_CBL_002L,"Electrical Power Line,4.16KV,From BUS-00B2-A t...",FCL_ELS_4.16KV_LINES,"Electrical Power Line,4.16KV",Air Handling Unit,Electrical Power Line,None,None,None
...,...,...,...,...,...,...,...,...,...
126013,YTF-VSTORE,Computer Storage Location,YTF EUS,Production & End User Support,None,None,None,None,None
126014,YX2411A,"Pump, Unwatering, Primary Tanks 1-2",TAB-PRM-P-SUMP,"P Bldg & Old PS Buildings, Primary Treatment S...","Pump,Centrifugal,single-Stage",Pump,None,None,None
126015,ZB20-CA-31,"Damper,Combustion Air, Steam Boiler #20",TAB-ACC-BO-0500,"Boiler #20, Volcano",Damper/Louver,Damper/Louver,None,None,None
126016,ZXDATA-PILOT1,"DO NOT USE - City of Toronto Pump, RW Pump 18",None,None,"Pump,Centrifugal",Pump,None,None,None


## SPARQL connection
The following will now load the data from the OWL file

In [13]:
from rdflib import Graph
import pandas as pd

# Load the RDFS .ttl file into an RDFLib Graph
g = Graph()
g.parse(r"../TWONTO/OWL/TWONTO.ttl", format="turtle")

# Prepare a custom SPARQL query
query = """

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX tw: <http://www.toronto.ca/TWONTO#>

SELECT DISTINCT ?label ?subject
       (COALESCE(?isEquivalentToCategory, "") AS ?isEquivalentToCategory)
       (COALESCE(?isEquivalentToClass, "") AS ?isEquivalentToClass)
       (COALESCE(?isSuperclassToCategory, "") AS ?isSuperclassToCategory)
       (COALESCE(?isSuperclassToClass, "") AS ?isSuperclassToClass)
       (COALESCE(?is_equivalent_to_tag_code, "") AS ?is_equivalent_to_tag_code)
       (COALESCE(?is_superclass_to_tag_code, "") AS ?is_superclass_to_tag_code)
WHERE {
  ?subject rdfs:label ?label .

#  FILTER NOT EXISTS { ?subject owl:deprecated true }

#  ?assetClass rdfs:label "asset" .
#  ?subject rdfs:subClassOf+ ?assetClass .

  OPTIONAL { ?subject tw:is_equivalent_to_Avantis_category ?isEquivalentToCategory . }
  OPTIONAL { ?subject tw:is_equivalent_to_Avantis_class ?isEquivalentToClass . }
  OPTIONAL { ?subject tw:is_superclass_to_Avantis_category ?isSuperclassToCategory . }
  OPTIONAL { ?subject tw:is_superclass_to_Avantis_class ?isSuperclassToClass . }
  OPTIONAL { ?subject tw:is_equivalent_to_tag_code ?is_equivalent_to_tag_code . }
  OPTIONAL { ?subject tw:is_superclass_to_tag_code ?is_superclass_to_tag_code . }
}
"""

# Execute the query
results = g.query(query)

# Convert results to a DataFrame, do not replace None with empty string
data = []
for row in results:
    data.append([
        str(row.label) if row.label else None,
        str(row.subject) if row.subject else None,
        str(row.isEquivalentToCategory) if row.isEquivalentToCategory else None,
        str(row.isEquivalentToClass) if row.isEquivalentToClass else None,
        str(row.isSuperclassToCategory) if row.isSuperclassToCategory else None,
        str(row.isSuperclassToClass) if row.isSuperclassToClass else None,
        str(row.is_equivalent_to_tag_code) if row.is_equivalent_to_tag_code else None,
        str(row.is_superclass_to_tag_code) if row.is_superclass_to_tag_code else None
    ])

df_sparql = pd.DataFrame(data, columns=[
    "label",
    "subject",
    "isEquivalentToCategory",
    "isEquivalentToClass",
    "isSuperclassToCategory",
    "isSuperclassToClass",
    "is_equivalent_to_tag_code",
    "is_superclass_to_tag_code"
])

df_sparql

,label,subject,isEquivalentToCategory,isEquivalentToClass,isSuperclassToCategory,isSuperclassToClass,is_equivalent_to_tag_code,is_superclass_to_tag_code
0,new_13040_code_proposed,http://www.toronto.ca/TWONTO#new_13040_code_pr...,None,None,None,None,None,None
1,contains,http://www.toronto.ca/TWONTO#00865,None,None,None,None,None,None
2,created,http://www.toronto.ca/TWONTO#00866,None,None,None,None,None,None
3,fully grounds,http://www.toronto.ca/TWONTO#00867,None,None,None,None,None,None
4,grounds,http://www.toronto.ca/TWONTO#00868,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
1323,steam,http://www.toronto.ca/TWONTO#00704,None,None,None,None,None,None
1324,thermal oil,http://www.toronto.ca/TWONTO#00745,None,None,None,None,None,None
1325,water,http://www.toronto.ca/TWONTO#00798,None,None,None,None,None,None
1326,water glycol mixture,http://www.toronto.ca/TWONTO#00799,None,None,None,None,None,None


Now to create the dictionary items for easy lookup later

In [14]:
dict_class = dict(zip(df_sparql['isEquivalentToClass'], df_sparql['label']))
dict_superclass = dict(zip(df_sparql['isSuperclassToClass'], df_sparql['label']))
dict_category = dict(zip(df_sparql['isEquivalentToCategory'], df_sparql['label']))
dict_supercategory = dict(zip(df_sparql['isSuperclassToCategory'], df_sparql['label']))
dict_tag = dict(zip(df_sparql['is_equivalent_to_tag_code'], df_sparql['label']))
dict_supertag = dict(zip(df_sparql['is_superclass_to_tag_code'], df_sparql['label']))
dict_iri = dict(zip(df_sparql['subject'], df_sparql['label']))
dict_class

{None: 'water-glycol mixture',
 'Vehicle': 'passenger vehicle',
 'Fan': 'fan',
 'Structure': 'structure',
 'Power Supply Unit': 'DC power supply',
 'Programmable Logic Control': 'PLC',
 'Remote Processor Unit': 'RPU panel',
 'Remote Transmission Unit': 'RPU panel',
 'Uninterruptible Power Supply': 'UPS',
 'Ultraviolet Disinfection': 'UV disinfection assembly',
 'Variable Frequency Drive': 'VFD',
 'Actuator': 'actuator',
 'Damper/Louver': 'damper',
 'Drier': 'air dryer',
 'Duct': 'air duct segment',
 'Vent': 'air duct segment',
 'Air Exchanger Unit': 'air exchange unit',
 'Air Handling Unit': 'air handler unit',
 'Scrubber': 'air scrubber',
 'Silencer': 'air silencer',
 'Alarm': 'alarm device',
 'Sampler': 'auto sampler',
 'Safety,Defibrillator': 'automatic external defibrillator',
 'Back Flow Preventer': 'backflow preventer',
 'Battery': 'battery',
 'Blower': 'blower',
 'Boat': 'boat',
 'Boiler': 'boiler',
 'Circuit Breaker': 'breaker',
 'Building': 'building',
 'Burner': 'burner',
 'C

## Manual Matching
Here we will load the manual matched file

In [15]:
df_manual = pd.read_excel('manualMatch.xlsx', sheet_name='LLM capability Test Dataset')
df_manual

,ID,Entity_number,Description,Parent,Category,Class,Suspended,Tag,Valid_Class,TH Suggestion,Parent_Description
0,104978.0,NX5232A,"Chiller, Silo Bldg Control Room",TAB-ACC-CI,"H.V.A.C.,Chiller",HVAC,NaN,NaN,http://www.toronto.ca/TWONTO#00213,NaN,Chiller and Air Coolers
1,119738.0,TAB-WA1-SQ-1984,"Lanyard 6 Ft- Velasco, Gabriel\t\t\t\t\t\t\t",TAB-WA1-SQ-0001,"PPE,Harness",Safety Equipment,NaN,SQ,http://www.toronto.ca/TWONTO#00333,NaN,WA1 Fall Arrest System
2,59232.0,THC-ACC-HTR-6025,"Heater, Unit, Electric, Heating System, Lower ...",THC-ELS-LP-4042A,"H.V.A.C.,Heater,Unit",HVAC,NaN,HTR,http://www.toronto.ca/TWONTO#00418,NaN,"Lighting Panel, Vortex Gallery, Headworks"
3,119966.0,TAB-WA4-SQ-3430,Fall Limiter - MFLT2/705F,TAB-WA4-SQ-0010,"PPE,Lanyard",Safety Equipment,NaN,SQ,http://www.toronto.ca/TWONTO#00341,NaN,WA4 Fall arrest systems
4,59390.0,THC-ACC-PDIT-6291,"Transmitter, Pressure Differential, Filter F-6...",THC-ACC-F-6291,"Transmitter,Pressure",HVAC,NaN,PDIT,http://www.toronto.ca/TWONTO#00605,NaN,"Filter, Outdoor Air Plenum, 2nd Floor, Mech Ro..."
...,...,...,...,...,...,...,...,...,...,...,...
44099,NaN,THC-PRM-TI-1204,"Temperature, Indicate, Oil Tank 1210",THC-PRM-T-1210,"Transmitter,Temperature",Transmitter,NaN,TI,http://www.toronto.ca/TWONTO#tujad-guzov-jilok...,NaN,NaN
44100,NaN,THC-PRM-TI-1214,"Temperature, Indicate, Oil Tank 1270",THC-PRM-T-1270,"Transmitter,Temperature",Transmitter,NaN,TI,http://www.toronto.ca/TWONTO#tujad-guzov-jilok...,NaN,NaN
44101,NaN,WTR-PPD-TI-1013,"Transmitter, Temperature, Indoor, AHU, Pump Ga...",WTR-PPD-AHU-1011,"Transmitter,Temperature",Indicator,NaN,TI,http://www.toronto.ca/TWONTO#tujad-guzov-jilok...,NaN,NaN
44102,NaN,WTR-PPD-TI-1014,"Transmitter, Temperature, Outdoor Air,",WTR-PPD-AHU-1011,"Transmitter,Temperature",Indicator,NaN,TI,http://www.toronto.ca/TWONTO#tujad-guzov-jilok...,NaN,NaN


In [16]:
dict_manual = dict(zip(df_manual['Entity_number'],df_manual['Valid_Class'].map(dict_iri)))
dict_manual

{'NX5232A': 'chiller',
 'TAB-WA1-SQ-1984': 'fall arrest lanyard',
 'THC-ACC-HTR-6025': 'space heater',
 'TAB-WA4-SQ-3430': 'fall restricting system',
 'THC-ACC-PDIT-6291': 'pressure transmitter',
 'THR-PLT-FSL-2222': 'flow switch',
 'TAB-WA1-SQ-1915': 'fall arrest harness',
 'THR-MCS-LD-9868': 'winch',
 'TAB-RSP-FSL-1221D': 'flow switch',
 'THR-WA1-LD-0051': 'synthetic web sling',
 'THR-AER-PSL-1915': 'pressure switch',
 'FHA-ELS-PDP-2100': '600V electrical panel',
 'FHO-SM-SPL-0005': 'spill kit',
 'COL-YKP-DA-0017S': nan,
 'THR-SES-SCBA-0520': 'portable compressed gas cylinder',
 'THC-MCS-SM-0240': 'welding machine',
 'TAB-ACC-FN-6433': 'fan',
 'THR-SES-SQ-9833S': nan,
 'TAB-ACC-TSL-1247': 'temperature switch',
 'FHA-SED-ZSH-0304F': 'limit switch',
 'FCL-ACC-FS-0403': 'flow switch',
 'THC-ELS-SQ-211160R': 'insulated glove',
 'TNT-PRM-LS-0504': 'level switch',
 'THC-ACC-HS-0119': 'manual switch',
 'THC-AER-F-7220': 'replaceable air filter',
 'THR-WA3-LD-0097S': nan,
 'THR-SES-PGM-35920

## Applying the Mapping
The mapping will now be applied, starting with the lest likely to be accurate to the most likely to be accurate

In [28]:
df['Valid_Class'] = None

df['Valid_Class'] = df['Category'].map(dict_supercategory).fillna(df['Valid_Class'])
print(f"After mapping Supercategory, {df['Valid_Class'].notna().sum()/df.shape[0]:.1%} mapped")
df['Valid_Class'] = df['Category'].map(dict_category).fillna(df['Valid_Class'])
print(f"After mapping Category, {df['Valid_Class'].notna().sum()/df.shape[0]:.1%} mapped, {df['Category'].map(dict_category).notna().sum()/df.shape[0]:.1%} changed")

df['Valid_Class'] = df['Extracted_Code'].map(dict_supertag).fillna(df['Valid_Class'])
print(f"After mapping Supertag, {df['Valid_Class'].notna().sum()/df.shape[0]:.1%} mapped, {df['Extracted_Code'].map(dict_supertag).notna().sum()/df.shape[0]:.1%} changed")
df['Valid_Class'] = df['Extracted_Code'].map(dict_tag).fillna(df['Valid_Class'])
print(f"After mapping Tag, {df['Valid_Class'].notna().sum()/df.shape[0]:.1%} mapped, {df['Extracted_Code'].map(dict_tag).notna().sum()/df.shape[0]:.1%} changed")

df['Valid_Class'] = df['Class'].map(dict_superclass).fillna(df['Valid_Class'])
print(f"After mapping SuperClass, {df['Valid_Class'].notna().sum()/df.shape[0]:.1%} mapped, {df['Class'].map(dict_superclass).notna().sum()/df.shape[0]:.1%} changed")
df['Valid_Class'] = df['Class'].map(dict_class).fillna(df['Valid_Class'])
print(f"After mapping Class, {df['Valid_Class'].notna().sum()/df.shape[0]:.1%} mapped, {df['Class'].map(dict_class).notna().sum()/df.shape[0]:.1%} changed")
df['Valid_Class'] = df['Entity_number'].map(dict_manual).fillna(df['Valid_Class'])

print(f"After mapping Manual Mapping, {df['Valid_Class'].notna().sum()/df.shape[0]:.1%} mapped, {df['Entity_number'].map(dict_manual).notna().sum()/df.shape[0]:.1%} changed")

df

After mapping Supercategory, 58.1% mapped
After mapping Category, 89.0% mapped, 57.0% changed
After mapping Supertag, 91.3% mapped, 31.0% changed
After mapping Tag, 98.1% mapped, 81.9% changed
After mapping SuperClass, 98.2% mapped, 8.8% changed
After mapping Class, 99.0% mapped, 83.6% changed
After mapping Manual Mapping, 100.0% mapped, 27.6% changed


,Entity_number,Description,Parent,Parent_Description,Category,Class,Suspended,Extracted_Code,Matched_Code,Valid_Class
0,\tFCL_ELS_CBL_001L,"Electrical Power Line,4.16KV,From BUS-00B1-A t...",FCL_ELS_4.16KV_LINES,"Electrical Power Line,4.16KV",Air Handling Unit,Electrical Power Line,None,None,None,cable segment
1,\tFCL_ELS_CBL_002D,"Electrical Power Line,4.16KV,From BUS-00B2-A t...",FCL_ELS_4.16KV_LINES,"Electrical Power Line,4.16KV",Air Handling Unit,Electrical Power Line,None,None,None,cable segment
2,\tFCL_ELS_CBL_002F,"Electrical Power Line,4.16KV,From BUS-00B2 to ...",FCL_ELS_4.16KV_LINES,"Electrical Power Line,4.16KV",Air Handling Unit,Electrical Power Line,None,None,None,cable segment
3,\tFCL_ELS_CBL_002H,"Electrical Power Line,4.16KV,From BUS-00B1 to ...",FCL_ELS_4.16KV_LINES,"Electrical Power Line,4.16KV",Air Handling Unit,Electrical Power Line,None,None,None,cable segment
4,\tFCL_ELS_CBL_002L,"Electrical Power Line,4.16KV,From BUS-00B2-A t...",FCL_ELS_4.16KV_LINES,"Electrical Power Line,4.16KV",Air Handling Unit,Electrical Power Line,None,None,None,cable segment
...,...,...,...,...,...,...,...,...,...,...
126013,YTF-VSTORE,Computer Storage Location,YTF EUS,Production & End User Support,None,None,None,None,None,defined collection of assets
126014,YX2411A,"Pump, Unwatering, Primary Tanks 1-2",TAB-PRM-P-SUMP,"P Bldg & Old PS Buildings, Primary Treatment S...","Pump,Centrifugal,single-Stage",Pump,None,None,None,pump
126015,ZB20-CA-31,"Damper,Combustion Air, Steam Boiler #20",TAB-ACC-BO-0500,"Boiler #20, Volcano",Damper/Louver,Damper/Louver,None,None,None,damper
126016,ZXDATA-PILOT1,"DO NOT USE - City of Toronto Pump, RW Pump 18",None,None,"Pump,Centrifugal",Pump,None,None,None,water-glycol mixture
